In [1]:
from func_def import *

In [2]:
events = get_links(url='http://www.ufcstats.com/statistics/events/completed?page=all')
events.date = pd.to_datetime(events.date)
events = events[events.date < get_current_date()].reset_index(drop=True)
events.to_csv('event_list.csv')
events

,date,event_url
0,2023-10-14,http://www.ufcstats.com/event-details/f3a07827...
1,2023-10-07,http://www.ufcstats.com/event-details/c8a49ff2...
2,2023-09-23,http://www.ufcstats.com/event-details/c945adc2...
3,2023-09-16,http://www.ufcstats.com/event-details/8fa2b065...
4,2023-09-09,http://www.ufcstats.com/event-details/ece28074...
...,...,...
661,1995-07-14,http://www.ufcstats.com/event-details/1c3f5e85...
662,1995-04-07,http://www.ufcstats.com/event-details/dedc3bb4...
663,1994-12-16,http://www.ufcstats.com/event-details/b60391da...
664,1994-09-09,http://www.ufcstats.com/event-details/1a49e067...


In [3]:
#get existing data
UPDATE_ALL = False

exist_data = []
exist_events_url = []

if not UPDATE_ALL:
    try:
        exist_data = pd.read_csv('data_pre_processed.csv',index_col=0)
        exist_events_url = list(np.unique(exist_data.event_url))
    except:
        print('Cant find data_pre_processed.csv')
        exist_data = []
        exist_events_url = []

print(f'Current Existing events: {len(exist_events_url)}')

Current Existing events: 652


In [4]:
#get new data
new_events = events[~events.event_url.isin(exist_events_url)].reset_index(drop=True)
#new_events = events
print(f'New events: {len(new_events)}')
new_events.head(3)

New events: 14


,date,event_url
0,2023-10-14,http://www.ufcstats.com/event-details/f3a07827...
1,2020-02-29,http://www.ufcstats.com/event-details/fc9a9559...
2,1998-10-16,http://www.ufcstats.com/event-details/32a3025d...


In [5]:
new_data = []
for index in tqdm(range(len(new_events))):
    try:
        new_data.extend(get_event_data(EVENT_URL=new_events.loc[index,'event_url'],EVENT_TIME=new_events.loc[index,'date']))
    except:
        continue   

try:
    data = pd.concat([exist_data,pd.DataFrame(new_data)]).reset_index(drop=True)
except:
    data = new_data

data[-3:-1]

100%|██████████| 14/14 [02:53<00:00, 12.39s/it]


,url,event_url,date,result,fighter,opponent,title,method,round,time,format,f_head_str,f_body_str,f_leg_str,f_distance_str,f_clinch_str,f_ground_str,o_head_str,o_body_str,o_leg_str,o_distance_str,o_clinch_str,o_ground_str,f_head_str_perc,f_body_str_perc,f_leg_str_perc,f_dist_str_perc,f_clinch_str_perc,f_ground_str_perc,o_head_str_perc,o_body_str_perc,o_leg_str_perc,o_dist_str_perc,o_clinch_str_perc,o_ground_str_perc
7259,http://www.ufcstats.com/fight-details/59861c3d...,http://www.ufcstats.com/event-details/fc9a9559...,2020-02-29 00:00:00,W,Jordan Griffin,TJ Brown,Featherweight Bout,Submission,2,3:38,3 Rnd (5-5-5),13of21,2of2,0of0,9of17,3of3,3of3,26of39,6of7,0of1,9of22,10of10,13of15,0.86,0.13,0.0,0.6,0.2,0.2,0.81,0.18,0.00,0.28,0.31,0.4
7260,http://www.ufcstats.com/fight-details/0bcb0416...,http://www.ufcstats.com/event-details/fc9a9559...,2020-02-29 00:00:00,L,Aalon Cru,Spike Carlyle,Featherweight Bout,KO/TKO,1,1:25,3 Rnd (5-5-5),2of4,0of4,0of4,2of12,0of0,0of0,19of27,0of1,1of1,2of6,4of5,14of18,1.00,0.00,0.0,1.0,0.0,0.0,0.95,0.00,0.05,0.10,0.20,0.7


In [7]:
pd.DataFrame(data).to_csv('data_pre_processed.csv',index=[0])